# Generative Pseudo Labeling for Domain Adaptation of Dense Retrievals

*Note: Adapted to Haystack from Nils Reimers' original [notebook](https://colab.research.google.com/gist/jamescalam/d2c888775c87f9882bb7c379a96adbc8/gpl-domain-adaptation.ipynb#scrollTo=183ff7ab)

The NLP models we use every day were trained on a corpus of data that reflects the world from the past. In the meantime, we've experienced world-changing events, like the COVID pandemics, and we'd like our models to know about them. Training a model from scratch is tedious work but what if we could just update the models with new data? Generative Pseudo Labeling comes to the rescue.

The example below shows you how to use GPL to fine-tune a model so that it can answer the query: "How is COVID-19 transmitted?".

We're using TAS-B: A DistilBERT model that achieves state-of-the-art performance on MS MARCO (500k queries from Bing Search Engine). Both DistilBERT and MS MARCO were created with data from 2018 and before, hence, it lacks the knowledge of any COVID-related information.

For this example, we're using just four documents. When you ask the model ""How is COVID-19 transmitted?", here are the answers that you get (dot-score and document):
- 94.84	Ebola is transmitted via direct contact with blood
- 92.87	HIV is transmitted via sex or sharing needles
- 92.31	Corona is transmitted via the air
- 91.54	Polio is transmitted via contaminated water or food


You can see that the correct document is only third, outranked by Ebola and HIV information. Let's see how we can make this better.

## Efficient Domain Adaptation with GPL
This notebook demonstrates [Generative Pseudo Labeling (GPL)](https://arxiv.org/abs/2112.07577), an efficient approach to adapt existing dense retrieval models to new domains and data.

We get a collection of 10k scientific papers on COVID-19 and then fine-tune the model within 15-60 minutes (depending on your GPU) so that it includes the COVID knowledge.

If we search again with the updated model, we get the search results we would expect:
- Query: How is COVID-19 transmitted
- 97.70	Corona is transmitted via the air
- 96.71	Ebola is transmitted via direct contact with blood
- 95.14	Polio is transmitted via contaminated water or food
- 94.13	HIV is transmitted via sex or sharing needles

### Prepare the Environment

#### Colab: Enable the GPU runtime
Make sure you enable the GPU runtime to experience decent speed in this tutorial.
**Runtime -> Change Runtime type -> Hardware accelerator -> GPU**

<img src="https://raw.githubusercontent.com/deepset-ai/haystack/main/docs/img/colab_gpu_runtime.jpg">


In [ ]:
!nvidia-smi

Mon Jan 30 14:59:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q datasets
!pip install "faiss-gpu>=1.6.3,<2"
!pip install -q git+https://github.com/deepset-ai/haystack.git
!pip install gwpy &> /dev/null

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Logging

We configure how logging messages should be displayed and which log level should be used before importing Haystack.
Example log message:
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/218_Olenna_Tyrell.txt
Default log level in basicConfig is WARNING so the explicit parameter is not necessary but can be changed easily:

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset

In [ ]:
# We load the TAS-B model, a state-of-the-art model trained on MS MARCO
max_seq_length = 350
model_name = "multi-qa-mpnet-base-dot-v1"

org_model = SentenceTransformer(model_name)
org_model.max_seq_length = max_seq_length

# Get Some Data on COVID-19
We select 10k scientific publications (title + abstract) that are connected to COVID-19. As a dataset, we use [TREC-COVID-19](https://huggingface.co/datasets/nreimers/trec-covid).

In [ ]:
!pip install wget
import wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
dev_data = wget.download('https://drive.google.com/uc?export=download&id=1bkYsb6KE7KmAxrUJupu3Q8kQyfBbTVMF')


In [ ]:
train_data = wget.download('https://drive.google.com/uc?export=download&id=1KFPlHpH69x_WHNKtM0TvMmyAe_zelvew')

In [ ]:
dataset = load_dataset("csv", data_files="/content/train_data.csv", split='train')
corpus = []
for row in dataset:
    text = row['Paragraph']
    corpus.append(text)
corpus = list(set(corpus))
print("Len Corpus:", len(corpus))

Len Corpus: 15555


In [ ]:
dataset = load_dataset("csv", data_files="/content/dev.csv", split='train')
corpus = []
for row in dataset:
    text = row['context']
    corpus.append(text)
corpus = list(set(corpus))
print("Len Corpus:", len(corpus))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2a01f9fe8ff3713b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.
Len Corpus: 1204


# Initialize Haystack Retriever and DocumentStore

Let's add corpus documents to `FAISSDocumentStore` and update corpus embeddings via `EmbeddingRetriever`

In [ ]:
!pip install -q gwpy

In [ ]:
from haystack.nodes.retriever import EmbeddingRetriever
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")
document_store.write_documents([{"content": t} for t in corpus])


retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="/content/adapted_retriever",
    model_format="sentence_transformers",
    max_seq_len=max_seq_length,
    progress_bar=True,
)
document_store.update_embeddings(retriever)

## (Optional) Download Pre-Generated Questions or Generate Them Outside of Haystack

The first step of the GPL algorithm requires us to generate questions for a given text passage. Even though our pre-COVID trained model hasn't seen any COVID-related content, it can still produce sensible queries by copying words from the input text. As generating questions from 10k documents is a bit slow (depending on the GPU used), we'll download question/document pairs directly from the Hugging Face hub.


In [ ]:
from tqdm.auto import tqdm

query_doc_pairs = []

load_queries_from_hub = False

# Generation of the queries is quite slow in Colab due to the old GPU and the limited CPU
# I pre-computed the queries and uploaded these to the HF dataset hub. Here we just download them
if load_queries_from_hub:
    generated_queries = load_dataset("nreimers/trec-covid-generated-queries", split="train")
    for row in generated_queries:
        query_doc_pairs.append({"question": row["query"], "document": row["doc"]})
else:
    # Load doc2query model
    t5_name = "t5-small"
    t5_tokenizer = AutoTokenizer.from_pretrained(t5_name)
    t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_name).cuda()

    batch_size = 32
    queries_per_doc = 3

    for start_idx in tqdm(range(0, len(corpus), batch_size)):
        corpus_batch = corpus[start_idx : start_idx + batch_size]
        enc_inp = t5_tokenizer(
            corpus_batch, max_length=max_seq_length, truncation=True, padding=True, return_tensors="pt"
        )

        outputs = t5_model.generate(
            input_ids=enc_inp["input_ids"].cuda(),
            attention_mask=enc_inp["attention_mask"].cuda(),
            max_length=64,
            do_sample=True,
            top_p=0.95,
            num_return_sequences=queries_per_doc,
        )

        decoded_output = t5_tokenizer.batch_decode(outputs, skip_special_tokens=True)

        for idx, query in enumerate(decoded_output):
            corpus_id = int(idx / queries_per_doc)
            query_doc_pairs.append({"question": query, "document": corpus_batch[corpus_id]})


print("Generated queries:", len(query_doc_pairs))

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


  0%|          | 0/38 [00:00<?, ?it/s]

Generated queries: 3612


# Use PseudoLabelGenerator to Genenerate Retriever Adaptation Training Data

PseudoLabelGenerator run will execute all three steps of the GPL [algorithm](https://github.com/UKPLab/gpl#how-does-gpl-work):
 1. Question generation - optional step
 2. Negative mining
 3. Pseudo labeling (margin scoring)

The output of the `PseudoLabelGenerator` is the training data we'll use to adapt our `EmbeddingRetriever`.


In [ ]:

from haystack.nodes.question_generator import QuestionGenerator
from haystack.nodes.label_generator import PseudoLabelGenerator

use_question_generator = True


if use_question_generator:
    questions_producer = QuestionGenerator(
        model_name_or_path="doc2query/msmarco-t5-base-v1",
        max_length=64,
        split_length=128,
        batch_size=32,
        num_queries_per_doc=3,
    )

else:
    questions_producer = query_doc_pairs

# We can use either QuestionGenerator or already generated questions in PseudoLabelGenerator
psg = PseudoLabelGenerator(questions_producer, retriever, max_questions_per_document=10, batch_size=32, top_k=10)
output, pipe_id = psg.run(documents=document_store.get_all_documents())

# Update the Retriever

Now that we have the generated training data produced by `PseudoLabelGenerator`, we'll update the `EmbeddingRetriever`. Let's take a peek at the training data.

In [ ]:
out = output.copy()

In [ ]:
import pickle

In [ ]:
with open('/content/filename.pickle', 'rb') as handle:
    out = (pickle.load(handle))

In [ ]:
with open('filename.pickle', 'wb') as handle:
    pickle.dump(output, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
out = open("/content/filename.pickle")

In [ ]:
output["gpl_labels"][50]

{'question': 'what does conservative force mean',
 'pos_doc': 'A conservative force that acts on a closed system has an associated mechanical work that allows energy to convert only between kinetic or potential forms. This means that for a closed system, the net mechanical energy is conserved whenever a conservative force acts on the system. The force, therefore, is related directly to the difference in potential energy between two different locations in space, and can be considered to be an artifact of the potential field in the same way that the direction and amount of a flow of water can be considered to be an artifact of the contour map of the elevation of an area.',
 'neg_doc': "Following the Peterloo massacre of 1819, poet Percy Shelley wrote the political poem The Mask of Anarchy later that year, that begins with the images of what he thought to be the unjust forms of authority of his time—and then imagines the stirrings of a new form of social action. It is perhaps the first mo

In [ ]:
len(output["gpl_labels"])

5009

In [ ]:
import inspect
print(inspect.signature(retriever.train))

(training_data: List[Dict[str, Any]], learning_rate: float = 2e-05, n_epochs: int = 1, num_warmup_steps: Union[int, NoneType] = None, batch_size: int = 16, train_loss: str = 'mnrl') -> None


In [ ]:
!pip install numba

from numba import cuda 
device = cuda.get_current_device()
device.reset()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!CUDA_LAUNCH_BLOCKING=1

In [ ]:
retriever.train(output["gpl_labels"], batch_size=4)

INFO:haystack.nodes.retriever._embedding_encoder:Training/adapting SentenceTransformer(
  (0): Transformer({'max_seq_length': 350, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
) with 5009 examples


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1252 [00:00<?, ?it/s]

## Verify that EmbeddingRetriever Is Adapted and Save It For Future Use

Let's repeat our query to see if the Retriever learned about COVID and can now rank it as #1 among the answers.

In [ ]:
print("Original Model")
show_examples(org_model)

print("\n\nAdapted Model")
show_examples(retriever.embedding_encoder.embedding_model)

Original Model
Query: How is COVID-19 transmitted
94.84	Ebola is transmitted via direct contact with blood
92.87	HIV is transmitted via sex or sharing needles
92.31	Corona is transmitted via the air
91.54	Polio is transmitted via contaminated water or food


Adapted Model
Query: How is COVID-19 transmitted
100.73	Corona is transmitted via the air
100.27	Ebola is transmitted via direct contact with blood
98.41	HIV is transmitted via sex or sharing needles
98.21	Polio is transmitted via contaminated water or food


In [ ]:
retriever.save("adapted_retriever")

In [ ]:
!apt install libomp-dev --quiet
!pip install faiss-gpu --quiet
!pip install --quiet sentencepiece --quiet
!pip install -U sentence-transformers --quiet
!pip install --upgrade pip --quiet
!pip install Pillow==9.0.0 --quiet
!pip install --quiet transformers --quiet
!pip install --quiet datasets --quiet
!pip install --quiet elasticsearch_dsl --quiet
!pip install optimum[intel] --quiet
!pip install evaluate --quiet
!pip install wget --quiet
!python -m pip install optimum[neural-compressor] --quiet
!python -m pip install optimum[openvino,nncf] --quiet

In [ ]:
PATH = '/content/adapted_retriever'
org_model = SentenceTransformer(PATH)
# tokenizer = BertTokenizer.from_pretrained(PATH, local_files_only=True)

In [ ]:
!wget -O eval_script.py "https://drive.google.com/uc?export=download&id=1GPBSpYDtpWoAUQkONxCjIXiqZqjL7ASZ&confirm=t&uuid=aabf1dfd-3fc2-4246-bd65-d544e8defd25&at=ALgDtswiYhx2DY4b8YGfnNIHgdWc:1674765375310"

In [ ]:
!python /content/eval_script.py \
      --data dev \
      --fraction 1 \
      --retriever_model PATH \
      --qna_model 'tachyon-11/xtremedistil-l6-h384-uncased-changed' \
      --onnx False \
      --query_expander 0 \
      --device cuda

In [ ]:
model = SentenceTransformer('/content/adapted_retriever')
top_k = 5

In [ ]:
import pandas as pd

In [ ]:
train_data = wget.download('https://drive.google.com/uc?export=download&id=1KFPlHpH69x_WHNKtM0TvMmyAe_zelvew')

In [ ]:
data = pd.read_csv("/content/dev.csv")

In [ ]:
data.head()

In [ ]:
import faiss

In [ ]:
# unique_themes = data['Theme'].unique()

# theme = unique_themes[0]
# paragraphs_for_theme = data[data['Theme'] == theme]['Paragraph'].unique()
# questions_for_theme = data[data['Theme'] == theme]['Question'].to_numpy()
questions = data['question'].to_numpy()
paragraphs = data['context'].unique()
questions_for_para = [data[data['context'] == para]['question'].to_numpy() for para in paragraphs]

para_for_ques = [[i, ] * len(p) for i, p in enumerate(questions_for_para)]
para_for_ques = sum(para_for_ques, [])

para_embeddings = model.encode(paragraphs)
para_embeddings.shape

d = para_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(para_embeddings)

In [ ]:
%%time
ques_embeddings = model.encode(questions)
D, I = index.search(ques_embeddings, 4)

CPU times: user 15.4 s, sys: 79.8 ms, total: 15.5 s
Wall time: 15.1 s


In [ ]:
topK_acc = 0.0
top1_acc = 0.0

for i, ind in enumerate(I):
  true_para = para_for_ques[i]
  topK_acc += (true_para in ind)
  top1_acc += (true_para == ind[0])

topK_acc = topK_acc / len(I)
top1_acc = top1_acc / len(I)

topK_acc, top1_acc

(0.732249642044976, 0.520424492546113)